In [3]:
using DifferentialEquations
using PyCall
import Random
using PyPlot
using Statistics
using BenchmarkTools
using Base.Threads
using SparseArrays
using StaticArrays

In [22]:
function KM_model!(du, u, p, t)

    @threads for i = 1:p[2]
        du[i] = p[3][i] + p[1] * sum(p[4][:, i] .* sin.(u .- u[i]))
    end

    return du
end

# sp = [K[1]/N, N, omega_0, s_adj, rows, values]

function sparse_KM_model1!(du, u, p, t)
    rows = p[5]
    @threads for j = 1:p[2]
        sumi = 0.0
        for i in nzrange(p[4], j)
            sumi +=  sin(u[rows[i]] - u[j])  #  p[6][p[5][i]] *
        end
        du[j] = p[3][j] + p[1] * sumi
    end

    return du
end;


function sparse_KM_model2!(du, u, p, t)
    rows = p[5]
    @threads for j = 1:p[2]
        sumi = sum(sin.(u[rows[nzrange(p[4], j)]] .- u[j]))
        du[j] = p[3][j] + p[1] * sumi
    end

    return du
end;

# list_p = [K[1]/N, N, omega_0, adjlist]

function adjlist_KM_model!(du, u, p, t)
    a = p[4]
    @threads for i = 1:p[2]
        du[i] = p[3][i] + p[1] * sum(sin.(u[a[i]] .- u[i]))
    end
    du
end


function order(theta)
    
    n = length(theta)
    
    real_R = 0.0
    imag_R = 0.0
    
    for i = 1:n
        real_R += cos(theta[i])
        imag_R += sin(theta[i])
    end
    
    real_R /= n
    imag_R /= n
    r = sqrt(real_R * real_R + imag_R * imag_R)
    
    return r
end


function adjmat_to_adjlist(A, threshold=1e-8)
    
    row, col = size(A)
    adjlist = Array{Int64}[]
    
    for i = 1:row
        tmp = []
        for j = 1:col
            if (abs(A[i, j]) > threshold)
                push!(tmp, j)
            end
        end
        push!(adjlist, tmp)
    end
    return adjlist
end

adjmat_to_adjlist (generic function with 2 methods)

In [23]:
Random.seed!(1234)

K = 0.7                      
N = 100                
dt = 0.05                  
T = 200.0                   
T_trans = 0.0                
mu = 2.0                     
sigma = 0.1 
p = 0.01
R = zeros(length(K))

nx = pyimport("networkx")
np = pyimport("numpy")

G = nx.gnp_random_graph(N, p, seed=1)
adj = nx.to_numpy_array(G, dtype=np.int)
adjlist = adjmat_to_adjlist(adj)
s_adj = sparse(adj)
rows = rowvals(s_adj)
# values = nonzeros(s_adj)

ind_transition = Int64(T_trans/dt)
theta_0 = rand(N) .* 2.0 .* pi .- pi
omega_0 = randn(N) .* sigma .+ mu 
tspan = (0.0, T)

p = [K[1]/N, N, omega_0, adj]
sp = [K[1]/N, N, omega_0, s_adj, rows]
list_p = [K[1]/N, N, omega_0, adjlist]

prob = ODEProblem(KM_model!, theta_0, tspan, p);
s_prob1 = ODEProblem(sparse_KM_model1!, theta_0, tspan, sp);
s_prob2 = ODEProblem(sparse_KM_model2!, theta_0, tspan, sp);
list_prob = ODEProblem(adjlist_KM_model!, theta_0, tspan, list_p);

In [24]:
function solving(prob, message)
    sol = solve(prob, Tsit5(), saveat=dt, abstol=1e-6, reltol=1e-6);
    nstep = length(sol.t)
    r = zeros(nstep)
    for j =1:nstep
        r[j] = order(sol.u[j])
    end
    println(mean(r))    
end

solving(prob, "non sparse")
solving(s_prob1, "sparse1")
solving(s_prob2, "sparse2")
solving(list_prob, "adjlist")

0.08980138313239366
0.0898013831323934
0.0898013831323934
0.0898013831323934


In [25]:
@btime solve(prob, Tsit5(), abstol=1e-6,reltol=1e-6);
@btime solve(s_prob1, Tsit5(), abstol=1e-6,reltol=1e-6);
@btime solve(s_prob2, Tsit5(), abstol=1e-6,reltol=1e-6);
@btime solve(list_prob, Tsit5(), abstol=1e-6,reltol=1e-6);

  52.413 ms (279234 allocations: 41.06 MiB)
  6.926 ms (240894 allocations: 4.80 MiB)
  17.564 ms (300534 allocations: 10.38 MiB)
  16.203 ms (257934 allocations: 7.83 MiB)


In [28]:
saveat = collect(0:0.1:T)
@btime solve(prob, Tsit5(), saveat=saveat, abstol=1e-6,reltol=1e-6);
@btime solve(s_prob1, Tsit5(), saveat=saveat, abstol=1e-6,reltol=1e-6);
@btime solve(s_prob2, Tsit5(), saveat=saveat, abstol=1e-6,reltol=1e-6);
@btime solve(list_prob, Tsit5(), saveat=saveat, abstol=1e-6,reltol=1e-6);

  53.498 ms (280902 allocations: 42.77 MiB)
  7.169 ms (242562 allocations: 6.50 MiB)
  18.113 ms (302202 allocations: 12.09 MiB)
  16.713 ms (259602 allocations: 9.53 MiB)


In [8]:
# p = 0.1 connectivity ====================

# serial
# 62.843 ms (345729 allocations: 53.59 MiB)
# 76.345 ms (3453465 allocations: 64.35 MiB)
# 44.622 ms (650811 allocations: 34.78 MiB)

# 4 threads
# 21.854 ms (349698 allocations: 54.13 MiB)
# 27.806 ms (3460759 allocations: 65.31 MiB)
# 18.513 ms (657717 allocations: 35.73 MiB)

# p = 0.01 connectivity====================
# 4 threads
# 17.801 ms (286109 allocations: 44.83 MiB)
# 7.224 ms (395329 allocations: 11.57 MiB)
# 11.786 ms (491097 allocations: 20.54 MiB)


In [5]:
nthreads()

4

In [172]:
function KM_model!(du, u, p, t)

    n = length(u)
    for i = 1:n
        sumj = 0.0
        for j = 1:n
            sumj += p[3][j, i] * sin(u[j] - u[i])
        end
        du[i] = p[2][i] + p[1] * sumj
    end

    return du
end;

In [7]:
using Base.Threads